# Y.Music

Este projeto tem como objetivo comparar as preferências musicais dos habitantes de Springfild e Shelbyville.

Foram estudadas três hipóteses:
1. A atividade dos usuários é diferente dependendo do dia da semana e da cidade. 
2. Durante as manhãs de segunda-feira, os moradores de Springfield e Shelbyville escutam diferentes gêneros. Isso também é verdadeiro para noites de sexta-feira. 
3. Os ouvintes de Springfield e Shelbyville têm diferentes preferências. Em Springfield, as pessoas preferem pop, enquanto Shelbyville tem mais fãs de rap.


## Visão geral dos dados <a id='data_review'></a>

In [2]:
import pandas as pd

In [3]:
# lendo o arquivo e armazenando em df
df = pd.read_csv('music_project_en.csv')

Exiba as primeiras 10 linhas da tabela:

In [4]:
# obtenha as 10 primeiras 10 linhas da tabela df
df.head(10)

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Shelbyville,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Springfield,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Shelbyville,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Shelbyville,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Springfield,08:34:34,Monday
5,842029A1,Chains,Obladaet,rusrap,Shelbyville,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Springfield,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Springfield,20:47:49,Wednesday
8,8FA1D3BE,L’estate,Julia Dalia,ruspop,Springfield,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Shelbyville,21:20:49,Wednesday


In [5]:
# obtendo informações gerais sobre os dados em df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63736 non-null  object
 2   artist    57512 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


A tabela contém sete colunas. Elas armazenam o mesmo tipo de dado: objetos.

De acordo com a documentação:
- `'userID'` — identificação do usuário
- `'Track'` — título da música
- `'artist'` — nome do artista
- `'genre'` — o gênero
- `'City'` — cidade do usuário
- `'time'` — tempo exato que a música foi tocada
- `'Day'` — dia da semana 
Nós podemos ver três problemas com o estilo nos nomes das colunas:
1. Alguns nomes estão com letra maiúscula, alguns com letra minúscula.
2. Há espaços em alguns nomes.
3. userId, além de conter espaço no início, tem letras maiúsculas e minúsculas, e não possuí underscore(_) para separar as palavras (snake_case).

A quantidade de valores das colunas é diferente. Isso significa que os dados contém valores ausentes.


### Conclusões <a id='data_review_conclusions'></a> 

Cada linha na tabela armazena dados sobre uma música que foi tocada. Algumas colunas descrevem a música por si só: seu título, artista e gênero. O restante contém informações sobre o usuário: a cidade de onde eles vêm, a quantidade de vezes que a música foi tocada. 

Está claro que os dados são suficientes para testar as hipóteses. Entretanto, há valores ausentes.

Para seguir adiante, precisamos pré-processar os dados.

## Pré-processamento dos dados <a id='data_preprocessing'></a>

### Estilo do cabeçalho <a id='header_style'></a>

In [6]:
df.columns

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

In [7]:
# renomeando colunas
df= df.rename(
    columns={
        '  userID' : 'user_id',
        'Track' : 'track',
        '  City  ' : 'city',
        'Day' : 'day'
    }
)

In [8]:
# checando o resultado: a lista dos nomes das colunas
df.columns

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

### Valores ausentes <a id='missing_values'></a>

In [9]:
# calculando valores ausentes
df.isna().sum()
df.isnull().sum()

user_id       0
track      1343
artist     7567
genre      1198
city          0
time          0
day           0
dtype: int64

Nem todos os valores ausentes afetam a pesquisa. Por exemplo, os valores ausentes na música e artista não é decisivo, por isso posso substituí-los por marcadores claros.

Mas os valores ausentes em 'genre' pode afetar a comparação de preferências musicais de Springfield e Shelbyville. Dessa forma, prossegui com a correção dos valores ausentes da seguinte forma:
* Preenchi os valores ausentes com marcadores
* Avaliei o quanto os valores ausentes podem afetar os cálculos

Substituir os valores ausentes em 'track', 'artist', e 'genre' com a string 'unknown'. Para fazer isso, crie a lista columns_to_replace, percorra-a com o ciclo for, e substitua os valores ausentes em cada uma das colunas:

In [10]:
# percorrendo os nomes das colunas e substituindo valores ausentes com 'unknown'
columns_to_replace = ['track', 'artist', 'genre']

for cols in columns_to_replace:
    df[cols]= df[cols].fillna('unknown')

In [11]:
# contando os valores ausentes
df.isna().sum()

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64

### Duplicatas <a id='duplicates'></a>

In [12]:
# contando duplicatas claras
df.duplicated().sum()

3826

In [13]:
# removendo duplicatas óbvias
df = df.drop_duplicates()

In [14]:
# verificando duplicatas
df.duplicated().sum()

0

Lidando com duplicatas não óbvias

In [15]:
# visualizando nomes de gêneros únicos
df['genre'].unique()

array(['rock', 'pop', 'folk', 'dance', 'rusrap', 'ruspop', 'world',
       'electronic', 'unknown', 'alternative', 'children', 'rnb', 'hip',
       'jazz', 'postrock', 'latin', 'classical', 'metal', 'reggae',
       'triphop', 'blues', 'instrumental', 'rusrock', 'dnb', 'türk',
       'post', 'country', 'psychedelic', 'conjazz', 'indie',
       'posthardcore', 'local', 'avantgarde', 'punk', 'videogame',
       'techno', 'house', 'christmas', 'melodic', 'caucasian',
       'reggaeton', 'soundtrack', 'singer', 'ska', 'salsa', 'ambient',
       'film', 'western', 'rap', 'beats', "hard'n'heavy", 'progmetal',
       'minimal', 'tropical', 'contemporary', 'new', 'soul', 'holiday',
       'german', 'jpop', 'spiritual', 'urban', 'gospel', 'nujazz',
       'folkmetal', 'trance', 'miscellaneous', 'anime', 'hardcore',
       'progressive', 'korean', 'numetal', 'vocal', 'estrada', 'tango',
       'loungeelectronic', 'classicmetal', 'dubstep', 'club', 'deep',
       'southern', 'black', 'folkrock', 

Duplicatas implícitas:
* hip
* hop
* hip-hop

Para corrigi-los declarei a função replace_wrong_genres() com dois parâmetros: 
* wrong_genres= — a lista de duplicatas
* correct_genre= — a string com o valor correto

A função deve corrigir os nomes na coluna 'genre' da tabela df, isto é, substituindo cada valor da lista wrong_genres por valores de correct_genre.

In [16]:
# função para substituir duplicatas implícitas
def replace_wrong_genres (wrong_genres, correct_genre):
    for wrong_genre in wrong_genres:
        df.loc[:,'genre'] = df.loc[:,'genre'].replace(wrong_genre, correct_genre)


In [17]:
# removendo duplicatas implícitas
duplicates = ['hip', 'hop', 'hip-hop']
correct_genre = 'hiphop'

replace_wrong_genres (duplicates, correct_genre)


In [18]:
# verificando valores duplicados
df['genre'].unique()

array(['rock', 'pop', 'folk', 'dance', 'rusrap', 'ruspop', 'world',
       'electronic', 'unknown', 'alternative', 'children', 'rnb',
       'hiphop', 'jazz', 'postrock', 'latin', 'classical', 'metal',
       'reggae', 'triphop', 'blues', 'instrumental', 'rusrock', 'dnb',
       'türk', 'post', 'country', 'psychedelic', 'conjazz', 'indie',
       'posthardcore', 'local', 'avantgarde', 'punk', 'videogame',
       'techno', 'house', 'christmas', 'melodic', 'caucasian',
       'reggaeton', 'soundtrack', 'singer', 'ska', 'salsa', 'ambient',
       'film', 'western', 'rap', 'beats', "hard'n'heavy", 'progmetal',
       'minimal', 'tropical', 'contemporary', 'new', 'soul', 'holiday',
       'german', 'jpop', 'spiritual', 'urban', 'gospel', 'nujazz',
       'folkmetal', 'trance', 'miscellaneous', 'anime', 'hardcore',
       'progressive', 'korean', 'numetal', 'vocal', 'estrada', 'tango',
       'loungeelectronic', 'classicmetal', 'dubstep', 'club', 'deep',
       'southern', 'black', 'folkrock

### Conclusões <a id='data_preprocessing_conclusions'></a>
Foram detectados três problemas com os dados:

- Estilo de cabeçalho incorreto
- Valores ausentes
- Duplicatas óbvias e implícitas

O cabeçalho foi limpo para fazer o processamento da tabela mais simples.

Todos os valores ausentes foram substituídos por 'unkown'. 

## Estudando hipóteses <a id='hypotheses'></a>

### Hipótese 1: comparando o comportamento dos usuários em duas cidades <a id='activity'></a>

De acordo com a primeira hipótese, usuários de Springfield and Shelbyville escutam música de forma diferente. Para isso usei os dados de três dias da semana: segunda-feira, quarta-feira, e sexta-feira.

In [19]:
# Contando as músicas tocadas em cada cidade
df.groupby('city')
df.groupby('city')['track'].count()

city
Shelbyville    18512
Springfield    42741
Name: track, dtype: int64

Springfield tem mais músicas tocadas do que Shelbyville. Mas isso não quer dizer que os cidadãos de Springfield escutam música com mais frequência. Essa cidade é somente maior, e tem mais usuários.

In [20]:
# Calculando as músicas escutadas em cada um desses três dias
df.groupby('day')['track'].count()

day
Friday       21840
Monday       21354
Wednesday    18059
Name: track, dtype: int64

Quarta-feira é o dia mais calmo em geral. Mas se considerarmos as duas cidades separadamente, devemos chegar a uma conclusão diferente.

In [21]:
# <criando a função number_tracks()>
def number_tracks(day, city):
    track_list = df[(df['city']== city) & (df['day']== day)]                                              
    track_list_count = track_list['user_id'].count()
    return track_list_count

In [22]:
# a quantidade de músicas tocadas em Springfield na segunda-feira
number_tracks('Monday', 'Springfield')

15740

In [23]:
# a quantidade de músicas tocadas em Shelbyville na segunda-feira
number_tracks('Monday', 'Shelbyville')

5614

In [24]:
# a quantidade de músicas tocadas em Springfield na quarta-feira
number_tracks('Wednesday', 'Springfield')

11056

In [25]:
# a quantidade de músicas tocadas em Shelbyville na quarta-feira
number_tracks('Wednesday', 'Shelbyville')

7003

In [26]:
# a quantidade de músicas tocadas em Springfield na sexta-feira
number_tracks('Friday', 'Springfield')

15945

In [27]:
# a quantidade de músicas tocadas em Shelbyville na sexta-feira
number_tracks('Friday', 'Shelbyville')

5895

In [28]:
# tabela com resultados
dados = [['Springfield', number_tracks('Monday', 'Springfield'), number_tracks('Wednesday', 'Springfield'),
          number_tracks('Friday', 'Springfield')], 
         ['Shelbyville', number_tracks('Monday', 'Shelbyville'), number_tracks('Wednesday', 'Shelbyville'),
          number_tracks('Friday', 'Shelbyville')]
        ]
colunas = ['city', 'monday', 'wednesday', 'friday']

tabela_resultado = pd.DataFrame(data=dados, columns=colunas)

tabela_resultado

,city,monday,wednesday,friday
0,Springfield,15740,11056,15945
1,Shelbyville,5614,7003,5895


**Conclusões**

Os dados revelam diferenças no comportamento dos usuários:

- Em Springfield, a quantidade de músicas tocadas tem seu auge nas segundas e sextas-feiras, enquanto na quarta-feira há uma diminuição na atividade.
- Em Shelbyville, ao contrário, usuários escutam mais música na quarta-feira. A atividade na segunda e sexta-feira é pequena.

Então a primeira hipótese parece ser correta.

### Hipótese 2: música no começo e no fim da semana <a id='week'></a>

De acordo com a segunda hipótese, na segunda-feira de manhã e sexta-feira à noite, habitantes de Springfield escutam gêneros que diferem de alguns usuários de Shelbyville gostam.

In [29]:
# obtendo a tabela spr_generala partir das linhas df,
# onde o valor na coluna 'city' é 'Springfield'
spr_general = df[df['city'] == 'Springfield']
spr_general


,user_id,track,artist,genre,city,time,day
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Springfield,14:07:09,Friday
4,E2DC1FAE,Soul People,Space Echo,dance,Springfield,08:34:34,Monday
6,4CB90AA5,True,Roman Messer,dance,Springfield,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Springfield,20:47:49,Wednesday
8,8FA1D3BE,L’estate,Julia Dalia,ruspop,Springfield,09:17:40,Friday
...,...,...,...,...,...,...,...
65073,83A474E7,I Worship Only What You Bleed,The Black Dahlia Murder,extrememetal,Springfield,21:07:12,Monday
65074,729CBB09,My Name,McLean,rnb,Springfield,13:32:28,Wednesday
65076,C5E3A0D5,Jalopiina,unknown,industrial,Springfield,20:09:26,Friday
65077,321D0506,Freight Train,Chas McDevitt,rock,Springfield,21:43:59,Friday


In [30]:
# obtendo os shel_general a partir das linhas df,
# onde os valores na coluna 'city' é Shelbyville'
shel_general = df[df['city'] == 'Shelbyville']
shel_general

,user_id,track,artist,genre,city,time,day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Shelbyville,20:28:33,Wednesday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Shelbyville,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Shelbyville,08:37:09,Monday
5,842029A1,Chains,Obladaet,rusrap,Shelbyville,13:09:41,Friday
9,E772D5C0,Pessimist,unknown,dance,Shelbyville,21:20:49,Wednesday
...,...,...,...,...,...,...,...
65063,D94F810B,Theme from the Walking Dead,Proyecto Halloween,film,Shelbyville,21:14:40,Monday
65064,BC8EC5CF,Red Lips: Gta (Rover Rework),Rover,electronic,Shelbyville,21:06:50,Monday
65065,29E04611,Bre Petrunko,Perunika Trio,world,Shelbyville,13:56:00,Monday
65066,1B91C621,(Hello) Cloud Mountain,sleepmakeswaves,postrock,Shelbyville,09:22:13,Monday


In [31]:
# criando função
def genre_weekday(dados, day, time1, time2):
    genre_df = dados[(dados['day'] == day) & (dados['time'] < time2) & (dados['time'] > time1)]
    genre_df_count = genre_df.groupby('genre')['track'].count()
    genre_df_sorted = genre_df_count.sort_values(ascending=False)
    return genre_df_sorted[:15]


In [32]:
# chamando a função para segunda-feira de manha em Springfield (use spr_general em vez de df table)
genre_weekday(spr_general, 'Monday', '06:00', '12:00')

genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
classical      157
metal          120
jazz           100
folk            97
soundtrack      95
Name: track, dtype: int64

In [33]:
# chamando a função para segunda-feira de manhã em Shelbyville (use shel_general em vez de df table)

genre_weekday(shel_general, 'Monday', '06:00', '12:00')

genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
world           36
rap             32
soundtrack      31
rnb             27
metal           27
Name: track, dtype: int64

In [34]:
# chamando a função para sexta-feira à tarde em Springfield
genre_weekday(spr_general, 'Friday', '13:00', '18:00')

genre
pop            664
dance          522
rock           471
electronic     442
hiphop         264
classical      239
alternative    189
ruspop         161
world          155
rusrap         151
jazz           150
metal           96
soundtrack      92
folk            84
rnb             82
Name: track, dtype: int64

In [35]:
# chamando a função para sexta-feira à tarde em Shelbyville
genre_weekday(shel_general, 'Friday', '13:00', '18:00')

genre
pop            281
dance          237
rock           221
electronic     173
hiphop         123
classical      106
alternative     83
ruspop          76
rusrap          59
world           57
jazz            45
folk            42
soundtrack      38
rnb             34
metal           34
Name: track, dtype: int64

**Conclusão**

Tendo comparado os 15 gêneros mais ouvidos na segunda-feira de manhã, nós podemos tirar as seguintes conclusões:

1. Usuários de Springfield e Shelbyville escutam músicas semelhantes. Os cinco gêneros mais ouvidos são os mesmos, apenas rock e música eletrônica trocaram de lugar.

2. Em Springfield, a quantidade de valores ausentes acabaram por serem tantos que o valor 'unknown' veio em 10º. Isso significa que valores ausentes tiveram uma considerável porção dos dados, que pode ser a base para questionamentos a confiabilidade das conclusões.

Para sexta-feira à tarde, a situação é parecida. Gêneros individuais variam um pouco, mas no todo, os 15 gêneros mais ouvidos são parecidos para as duas cidades.

Assim, a segunda hipótese foi parcialmente provada:
* Usuários escutam gêneros musicais parecidos no começo e no fim da semana.
* Não há grande diferença entre Springfield e Shelbyville. Nas duas cidades, pop é o gênero mais popular.

No entanto, o número de valores ausentes faz esse resultado ser questionável. Em Springfield, há tantos que eles afetaram o top 15. Se não nos faltassem esses valores, as coisas poderiam ser diferentes.

### Hipótese 3: preferências em Springfield e Shelbyville <a id='genre'></a>

Hipótese: Shelbyville ama rap. Cidadãos de Springfield curtem mais pop.

In [36]:
spr_genres = spr_general.groupby('genre')['track'].count().sort_values(ascending=False)

In [38]:
# exibindo as primeiras 10 linhas de spr_genres
spr_genres.head(10)

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: track, dtype: int64

In [39]:
shel_genres = shel_general.groupby('genre')['track'].count().sort_values(ascending=False)

In [40]:
# exibindo as 10 primeiras linhas de shel_genres
shel_genres.head(10)

genre
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: track, dtype: int64

**Conclusão**

A hipótese foi parcialmente provada:
* Música pop é o gênero mais em Springfield, como esperado.
* Entretanto, música pop acabou por ser igualmente popular em Springfield e Shelbyville, e rap não estava no top 5 em nenhuma cidade.


# Conclusões <a id='end'></a>

Foram observadas as três hipóteses seguintes:

1. A atividade do usuário varia dependendo do dia da semana e da cidade. 
2. Nas segundas-feiras de manhã, os habitantes de Springfield e Shelbyville escutam diferentes gêneros. Isso também é verdadeiro para noites de sexta-feira. 
3. Ouvintes de Springfield e Shelbyville têm preferências diferentes. Tanto em Springfield como Shellbyville, eles preferem pop.

Depois de analisar os dados, concluí que:

1. A atividade do usuário em Springfield e Shelbyville depende do dia da semana, embora as cidades variam de formas diferentes. 

A primeira hipótese é totalmente aceita.

2. As preferências musicais não variam significativamente ao decorrer da semana tanto em Springfield como em Shelbyville. É possível ver pequenas diferenças na ordem nas segundas-feiras, mas:
* Em Springfield e Shelbyville, as pessoas escutam mais música pop.

Então nós podemos aceitar essa hipótese. Nós devemos também ter em mente que o resultado pode ter sido diferente se não fosse pelos valores ausentes.

3. Acontece que preferências musicais dos usuários de Springfield e Shelbyville são bastante parecidas.

A terceira hipótese foi rejeitada. Se há alguma diferença nas preferências, ela não pode ser vista nesses dados.

### Observação
Em projetos reais, pesquisas envolvendo teste estatístico de hipóteses, que é mais preciso e mais quantitativo. Também perceba que você não pode sempre tirar conclusões sobre uma cidade inteira com base em dados de apenas uma fonte.